In [1]:
import json
import pandas as pd 
import torch

TRAIN_DATA = "/kaggle/input/data-json/data/train.json"

def generate_joint_data(file_path):
    df = pd.read_json(file_path)
    df = df.transpose()

    #Extract the claims and NLI labels (Entailment/Contradiction).
    claims = df.Statement.tolist()
    nli_labels = df.Label.tolist()

    primary_indices = df.Primary_evidence_index.tolist()
    secondary_indices = df.Secondary_evidence_index.tolist()
    primary_cts = df.Primary_id.tolist()
    secondary_cts = df.Secondary_id.tolist()
    types = df.Type.tolist()
    sections = df.Section_id.tolist()

    primary_evidence_sentences = list()
    secondary_evidence_sentences = list()

    #Process the clinical trial report files.
    for idx in range(len(claims)):
        file_name = "/kaggle/input/data-json/data/CTs/" + primary_cts[idx] + ".json"

        with open(file_name, 'r') as f:
            data = json.load(f)
            primary_evidence_sentences.append(data[sections[idx]])

        if types[idx] == "Comparison":
            file_name = "/kaggle/input/data-json/data/CTs/" + secondary_cts[idx] + ".json"
            with open(file_name, 'r') as f:
                data = json.load(f)
                secondary_evidence_sentences.append(data[sections[idx]])
        else:
            secondary_evidence_sentences.append(list())

    #Generate the joint data instances and labels of evidence sentences. 
    joint_data = list()
    evidence_labels =  list()
    for claim_id in range(len(claims)):
        claim = claims[claim_id]
        current_evidence_labels = list()

        primary_sents = primary_evidence_sentences[claim_id]
        full_instance = claim + " [SEP] "
        
        #One data instance is a concatenation of the claim and all sentences from the CTR.
        # "claim [SEP] candidate_sent_1 [SEP] candidate_sent_2 (...) [SEP] candidate_sent_n"
        for sid in range(len(primary_sents)):
            candidate_sentence = primary_sents[sid]
            full_instance += candidate_sentence
            full_instance += " [SEP] "
            current_evidence_labels.append(1 if sid in primary_indices[claim_id] else 0)

        if types[claim_id] == "Comparison":        
            secondary_sents = secondary_evidence_sentences[claim_id]
            for sid in range(len(secondary_sents)):
                candidate_sentence = secondary_sents[sid]
                full_instance += candidate_sentence
                full_instance += " [SEP] "
                current_evidence_labels.append(1 if sid in secondary_indices[claim_id] else 0)
        joint_data.append(full_instance)

        #For a given clinical trial, evidence string is a string of 0's and 1's,
        # denoting whether the i-th candidate sentence is or is not evidence.
        evidence_string = ""
        for lab in current_evidence_labels:
            evidence_string += str(int(lab))
        evidence_labels.append(evidence_string)

    return joint_data, nli_labels, evidence_labels

    import torch
import torch

def generate_masks(encoded):
    """
    Generate masks for sentences in the encoded data. This function assumes
    that sentences are separated by a specific separator token, such as the
    [SEP] token in BERT models. The mask indicates the presence of actual
    data versus padding or separator tokens. Ensures each mask is at least 2D.

    :param encoded: Encoded representation of the input data, including input_ids.
    :return: A list of 2D mask tensors corresponding to each input in the batch, indicating
             valid tokens for attention mechanisms.
    """
    sep_token_id = 102  # Assuming BERT's [SEP] token ID, adjust if using a different model/tokenizer.
    masks = []

    for input_ids in encoded['input_ids']:
        # Initialize a 2D list to build the mask.
        mask_2d = []

        # Find positions of the separator tokens.
        sep_positions = (input_ids == sep_token_id).nonzero(as_tuple=True)[0]

        # Ensure at least one separator token is present for processing.
        if len(sep_positions) == 0:
            # If no separator token is present, treat the whole sequence as a single sentence.
            mask_2d.append(torch.ones_like(input_ids, dtype=torch.long).tolist())
        else:
            start_pos = 0
            for pos in sep_positions:
                # Mark positions between separators (inclusive of start, exclusive of end) as valid.
                sentence_mask = [0] * len(input_ids)
                sentence_mask[start_pos:pos.item()] = [1] * (pos - start_pos).item()
                mask_2d.append(sentence_mask)
                start_pos = pos.item() + 1

        # Convert the 2D list to a tensor, ensuring it has at least two dimensions.
        mask_tensor = torch.tensor(mask_2d, dtype=torch.long)
        masks.append(mask_tensor)

    return masks

class CtDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, stances):
        self.encoded = encodings
        self.labels = labels
        self.stances = stances

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
        item['labels'] = self.labels[idx]
        item['stance'] = self.stances[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, hidden_dim, n_labels, hidden_dropout_prob = 0.1):
        super().__init__()
        self.dense = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.out_proj = nn.Linear(hidden_dim, n_labels)

    def forward(self, x, **kwargs):
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

#Applies a linear weighting / self-attention layer.
class WordAttention(nn.Module):
    """
    x: (BATCH_SIZE, N_sentence, N_token, INPUT_SIZE)
    token_mask: (batch_size, N_sep, N_token)
    out: (BATCH_SIZE, N_sentence, INPUT_SIZE)
    mask: (BATCH_SIZE, N_sentence)
    """
    def __init__(self, INPUT_SIZE, PROJ_SIZE, dropout = 0.0):
        super(WordAttention, self).__init__()

        self.activation = torch.tanh
        self.att_proj = nn.Linear(INPUT_SIZE, PROJ_SIZE)
        self.dropout = nn.Dropout(dropout)
        self.att_scorer = nn.Linear(PROJ_SIZE, 1)
        
    def forward(self, x, token_mask):
        proj_input = self.att_proj(self.dropout(x.view(-1, x.size(-1))))
        proj_input = self.dropout(self.activation(proj_input))
        raw_att_scores = self.att_scorer(proj_input).squeeze(-1).view(x.size(0),x.size(1),x.size(2)) # (Batch_size, N_sentence, N_token)
        att_scores = F.softmax(raw_att_scores.masked_fill((1 - token_mask).bool(), float('-inf')), dim=-1)
        att_scores = torch.where(torch.isnan(att_scores), torch.zeros_like(att_scores), att_scores) # Replace NaN with 0
        batch_att_scores = att_scores.view(-1, att_scores.size(-1)) # (Batch_size * N_sentence, N_token)
        out = torch.bmm(batch_att_scores.unsqueeze(1), x.view(-1, x.size(2), x.size(3))).squeeze(1) 
        # (Batch_size * N_sentence, INPUT_SIZE)
        out = out.view(x.size(0), x.size(1), x.size(-1))
        mask = token_mask[:,:,0]
        return out, mask

class DynamicSentenceAttention(nn.Module):
    """
    input: (BATCH_SIZE, N_sentence, INPUT_SIZE)
    output: (BATCH_SIZE, INPUT_SIZE)
    """
    def __init__(self, INPUT_SIZE, PROJ_SIZE, REC_HID_SIZE = None, dropout = 0.1):
        super(DynamicSentenceAttention, self).__init__()
        self.activation = torch.tanh
        self.att_proj = nn.Linear(INPUT_SIZE, PROJ_SIZE)
        self.dropout = nn.Dropout(dropout)
        
        if REC_HID_SIZE is not None:
            self.contextualized = True
            self.lstm = nn.LSTM(PROJ_SIZE, REC_HID_SIZE, bidirectional = False, batch_first = True)
            self.att_scorer = nn.Linear(REC_HID_SIZE, 2)
        else:
            self.contextualized = False
            self.att_scorer = nn.Linear(PROJ_SIZE, 2)
        
    def forward(self, sentence_reps, sentence_mask, att_scores, valid_scores):
        sentence_mask = torch.logical_and(sentence_mask, valid_scores)
        

        if sentence_reps.size(0) > 0:
            att_scores = F.softmax(att_scores.masked_fill((~sentence_mask).bool(), -1e4), dim=-1)
            result = torch.bmm(att_scores.unsqueeze(1), sentence_reps).squeeze(1)
            return result 
        else:
            return sentence_reps[:,0,:]


#The final joint model used for the tasks.
class ModelForSequenceClassification(nn.Module):
    def __init__(self, base_model, hidden_dim=1024, n_labels=2):
        super().__init__()
        
        #DeBERTa-v3-large hidden size iz 1024.
        #We use DeBERTa as the base model for encoding the data instances.
        self.deberta = base_model
        
        self.word_attention = WordAttention(hidden_dim, hidden_dim, dropout=0.1)
        self.evidence_linear = ClassificationHead(hidden_dim=hidden_dim, 
                                        n_labels=n_labels, hidden_dropout_prob=0.1)
        
        self.evidence_criterion = nn.CrossEntropyLoss(ignore_index=2)
        self.nli_criterion = nn.CrossEntropyLoss()
        
        self.sentence_attention = DynamicSentenceAttention(hidden_dim, hidden_dim, dropout=0.1)
        self.nli_linear = ClassificationHead(hidden_dim, 3, hidden_dropout_prob = 0.1)
        
        self.extra_modules = [
            self.sentence_attention,
            self.nli_linear,
            self.evidence_linear,
            self.nli_criterion,
            self.evidence_criterion,
            self.word_attention
        ]            
   
    def select_valid(self, token_reps, token_mask, valid_sentences):
        
        if valid_sentences.size(1) > token_reps[:,1:,:,:].size(1):
            valid_sentences = valid_sentences[:, :token_reps[:,1:,:,:].size(1)]
               
        evidence_reps = token_reps[:,1:,:,:][valid_sentences]
        evidence_token_mask = token_mask[:,1:,:][valid_sentences]
        evidence_reps = evidence_reps.view(1, evidence_reps.size(0), evidence_reps.size(1), evidence_reps.size(2))
        evidence_token_mask = evidence_token_mask.view(1, evidence_token_mask.size(0), evidence_token_mask.size(1))
        if len(evidence_reps.shape) == 3 or evidence_reps.size(1) == 0:
            evidence_reps = token_reps[:,1,:,:].unsqueeze(1) # First sentence is claim; second is dummy
            evidence_token_mask = token_mask[:,1,:].unsqueeze(1)
        return evidence_reps, evidence_token_mask
        
        
    def forward(
        self,
        encoded,
        attention_mask,
        nli_label, 
        evidence_label,
        sample_p = 1,
        #return_features=True,
        **kwargs
    ):
        
        transformation_indices = token_idx_by_sentence(encoded, 2, "bert")
#         print("transformation indices", transformation_indices)
        transformation_indices = [torch.tensor(tensor, device=device) for tensor in transformation_indices]
#         print("tranformation indices device", transformation_indices)
        batch_indices, indices_by_batch, mask = transformation_indices # (batch_size, N_sep, N_token)


        deberta_out = self.deberta(encoded, attention_mask)[0] # (BATCH_SIZE, sequence_len, BERT_DIM)
        deberta_tokens = deberta_out[batch_indices, indices_by_batch, :]
    
        #represent sentences as weighted self-attention reps
#         mask = mask.to(device)
        sentence_reps, sentence_mask = self.word_attention(deberta_tokens, mask)         
        
        #logits of linear predictor
        evidence_out = self.evidence_linear(sentence_reps)      
        
        ## New linear
        att_scores = evidence_out[:,:,1] # (BATCH_SIZE, N_sentence)
         
        if (torch.rand(1, device=device) < sample_p).all():
            valid_scores = evidence_out[:,:,1] > evidence_out[:,:,0]
        else:
            valid_scores = evidence_label == 1 # Ground truth
#             valid_scores = valid_scores[:mask.size(1)]
            valid_scores = valid_scores[:,:mask.size(1)]

        
        paragraph_rep = self.sentence_attention(sentence_reps, sentence_mask, att_scores, valid_scores) 
        # (BATCH_SIZE, BERT_DIM) 
        
        nli_out = self.nli_linear(paragraph_rep) # (Batch_size, 3)
        
        #for loss calculation
        if evidence_label.size(1) > evidence_out.size(1):
            evidence_label = evidence_label[:,:evidence_out.size(1)]
        evidence_loss = self.evidence_criterion(evidence_out.view(-1, 2), 
                                                      evidence_label.reshape(-1)) # ignore index 2
        
        evidence_preds = (torch.softmax(evidence_out, dim=1)[:, 1] > 0.5).nonzero().flatten()
        
        nli_loss = self.nli_criterion(nli_out, nli_label)
        nli_out = torch.argmax(nli_out.cpu(), dim=-1).detach().numpy().tolist()
            
        return evidence_out, evidence_preds, evidence_loss, nli_out, nli_loss
    
    
    def evaluate(
        self,
        encoded,
        attention_mask,
        transformation_indices,
        **kwargs
    ):
        batch_indices, indices_by_batch, mask = transformation_indices # (batch_size, N_sep, N_token)
       
        deberta_out = self.deberta(encoded, attention_mask)[0] # (BATCH_SIZE, sequence_len, BERT_DIM)
        deberta_tokens = deberta_out[batch_indices, indices_by_batch, :]
    
        #represent sentences as weighted self-attention reps
        sentence_reps, sentence_mask = self.word_attention(deberta_tokens, mask)         
        
        #logits of linear predictor
        evidence_out = self.evidence_linear(sentence_reps)      
        
        att_scores = evidence_out[:,:,1] # (BATCH_SIZE, N_sentence)    
        valid_scores = evidence_out[:,:,1] > evidence_out[:,:,0]
        
        paragraph_rep = self.sentence_attention(sentence_reps, sentence_mask, att_scores, valid_scores) 
        # (BATCH_SIZE, BERT_DIM) 
        
        evidence_preds = (torch.softmax(evidence_out, dim=1)[:, 1] > 0.5).nonzero().flatten()

        nli_out = self.nli_linear(paragraph_rep) # (Batch_size, 3)        
        nli_out = torch.argmax(nli_out.cpu(), dim=-1).detach().numpy().tolist()
        print("---------------------in fucntion eval ",nli_out)
            
        self.deberta.train()
        return evidence_out, evidence_preds, nli_out
    

In [3]:

import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup, AdamW

# from model import ModelForSequenceClassification
# from prepare_joint import generate_joint_data, generate_masks

TRAIN_PATH = "/kaggle/input/data-json/data/train.json"
DEV_PATH = "/kaggle/input/data-json/data/dev.json"
TEST_PATH = "/kaggle/input/data-json/data/test.json"

#DeBERTa-v3-large, additionally fine-tuned on popular NLI datasets.
DEBERTA_PATH = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
# DEBERTA_PATH = "dmis-lab/biobert-base-cased-v1.2"
device = torch.device('cuda:0')

'''
Torch dataset used for the model. 

encoded: DeBERTa-encoded representation of a training instance (claim + all sentences)
labels: evidence labels
nlis: NLI/entailment labels 
'''
class CtDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, nlis):
        self.encoded = encodings
        self.labels = labels
        self.nlis = nlis

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
        item['labels'] = self.labels[idx]
        item['nli'] = self.nlis[idx]
        return item

    def __len__(self):
        return len(self.labels)     

def batch_evidence_label(labels, padding_idx = 2):
    max_sent_len = max([len(label) for label in labels])
    label_matrix = torch.ones(len(labels), max_sent_len) * padding_idx
    label_list = []
    for i, label in enumerate(labels):
        for j, evid in enumerate(label):
            label_matrix[i,j] = int(evid)
        label_list.append([int(evid) for evid in label])
    return label_matrix.long(), label_list


def batch_sentence_mask(masks):
    batch_mask = masks
    padded_batch_mask = list()
    max_shape = -1
    for m in batch_mask:
        if m.size(0) > max_shape:
            max_shape = m.size(0)

    padded_batch_mask = list()
    for m in batch_mask:
        if m.size(0) < max_shape:
            expanded = torch.cat((m, torch.zeros((max_shape - m.size(0), m.size(1)))))
        else:
            expanded = m

        expanded = expanded.view(1, expanded.size(0), expanded.size(1))
        padded_batch_mask.append(expanded)

    padded_batch_mask = torch.cat(padded_batch_mask)
    return padded_batch_mask
    

def token_idx_by_sentence(input_ids, sep_token_id, model_name):
    """
    Compute the token indices matrix of the BERT output.
    input_ids: (batch_size, paragraph_len)
    batch_indices, indices_by_batch, mask: (batch_size, N_sentence, N_token)
    bert_out: (batch_size, paragraph_len,BERT_dim)
    bert_out[batch_indices,indices_by_batch,:]: (batch_size, N_sentence, N_token, BERT_dim)
    """
    padding_idx = -1
    input_ids = input_ids.cpu()
    sep_tokens = (input_ids == sep_token_id).bool()
    paragraph_lens = torch.sum(sep_tokens,1).numpy().tolist()
    indices = torch.arange(sep_tokens.size(-1)).unsqueeze(0).expand(sep_tokens.size(0),-1)
    sep_indices = torch.split(indices[sep_tokens],paragraph_lens)
    paragraph_lens = []
    all_word_indices = []
    for paragraph in sep_indices:
        if "large" in model_name:
            paragraph = paragraph[1:]
        word_indices = [torch.arange(paragraph[i]+1, paragraph[i+1]+1) for i in range(paragraph.size(0)-2)]
        paragraph_lens.append(len(word_indices))
        all_word_indices.extend(word_indices)

    indices_by_sentence = nn.utils.rnn.pad_sequence(all_word_indices, batch_first=True, padding_value=padding_idx)
    indices_by_sentence_split = torch.split(indices_by_sentence,paragraph_lens)
    indices_by_batch = nn.utils.rnn.pad_sequence(indices_by_sentence_split, batch_first=True, padding_value=padding_idx)
    batch_indices = torch.arange(sep_tokens.size(0)).unsqueeze(-1).unsqueeze(-1).expand(-1,indices_by_batch.size(1),indices_by_batch.size(-1))
    mask = (indices_by_batch>=0) 

    return batch_indices.long(), indices_by_batch.long(), mask.long()

#Function for evaluating the model output.
def evaluation(model, dataset, data_masks):
    model.eval()
    evidence_predictions = list()
    evidence_labels = list()
    nli_preds = list()
    nli_labels = list()
    batch_size = 4
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(DataLoader(dataset, batch_size = 4, shuffle=False))):    
    
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
#             labels = batch['labels']
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            padded_evidence_label, evidence_label = batch_evidence_label(batch["labels"], padding_idx = 2)
            sentence_masks = batch_sentence_mask(data_masks[i*batch_size:i*batch_size+batch_size])
            sentence_masks = sentence_masks.to(device)

            nli_label = batch["nli"]
            nli_label = list(map(lambda x : 1 if x == "Entailment" else 0, nli_label))
            nli_label = np.asarray(nli_label)
            nli_label = torch.from_numpy(nli_label.astype('long'))
            nli_label = nli_label.to(device)
            
            evidence_label = padded_evidence_label.to(device)

            
        
            evidence_out, evidence_preds, evidence_loss, nli_out, nli_loss = \
            model(input_ids, attention_mask,nli_label, evidence_label, sentence_masks= sentence_masks)

            batch_labels = batch["labels"]
            
            batch_selected = (torch.softmax(evidence_out, dim=2)[:,:,1] > 0.5).tolist()
            for idx in range(len(batch_selected)):
                selected = [1 if l else 0 for l in batch_selected[idx]]
                evidence_predictions.extend(selected)
                
                true = [1 if c=="1" else 0 for c in batch_labels[idx]]
                evidence_labels.extend(true)
                
                if len(evidence_labels) > len(evidence_predictions):
                    miss = len(evidence_labels) - len(evidence_predictions)
                    evidence_predictions.extend([0] * miss)
                elif len(evidence_labels) < len(evidence_predictions):
                    miss = len(evidence_predictions) - len(evidence_labels) 
                    evidence_labels.extend([0] * miss)
                    
               
            nli_labels.extend(nli_label.cpu().numpy().tolist())
            nli_preds.extend(nli_out)
    print(nli_labels, nli_preds)

    nli_f1 = f1_score(nli_labels,nli_preds, average="macro")
    nli_precision = precision_score(nli_labels,nli_preds,average="macro")
    nli_recall = recall_score(nli_labels,nli_preds,average="macro")
    
    #print(evidence_predictions)
    evidence_f1 = f1_score(evidence_labels,evidence_predictions,average="macro")
    evidence_precision = precision_score(evidence_labels,evidence_predictions,average="macro")
    evidence_recall = recall_score(evidence_labels,evidence_predictions,average="macro")
    return nli_f1, nli_precision, nli_recall, evidence_f1, evidence_precision, evidence_recall
    

#Main training loop.
def train():
    #Load the base model.
    deberta = AutoModel.from_pretrained(DEBERTA_PATH)
    deberta = deberta.to(device)

    #Instantiate the developed model.
    model = ModelForSequenceClassification(deberta)
    model.to(device)
    settings = [{'params': model.deberta.parameters(), 'lr': 1e-5}]
    for module in model.extra_modules:
        settings.append({'params': module.parameters(), 'lr': 5e-6})
        
    #Load the tokenizer.
    tokenizer = AutoTokenizer.from_pretrained(DEBERTA_PATH, model_max_length=512)

    #Prepare and generate all data for the model.
    joint_train, nli_labels_train, evidence_labels_train = generate_joint_data(TRAIN_PATH)
    joint_dev, nli_labels_dev, evidence_labels_dev = generate_joint_data(DEV_PATH)

    encoded_train = tokenizer(joint_train, return_tensors='pt',
                        truncation_strategy='only_first', add_special_tokens=True, padding=True)
    encoded_dev = tokenizer(joint_dev, return_tensors='pt',
                        truncation_strategy='only_first', add_special_tokens=True, padding=True)
    train_masks = generate_masks(encoded_train)
    dev_masks = generate_masks(encoded_dev)

    train_dataset = CtDataset(encoded_train, evidence_labels_train, nli_labels_train)
    dev_dataset = CtDataset(encoded_dev, evidence_labels_dev, nli_labels_dev)

    optimizer = torch.optim.AdamW(settings)
    model.train()

    #Hyperparameters.
    epochs = 20
    batch_size = 3
    update_step = 200
    NUM_ACCUMULATION_STEPS = 4
    prev_performance = 0
    scheduler = get_cosine_schedule_with_warmup(optimizer, 0, epochs)

    #Main training loop.
    for epoch in range(epochs):
        model.train()

        tq = tqdm(DataLoader(train_dataset, batch_size=batch_size, shuffle=False))
        for i, batch in enumerate(tq):
            optimizer.zero_grad()

            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            padded_evidence_label, evidence_label = batch_evidence_label(batch["labels"], padding_idx = 2)
            sentence_masks = batch_sentence_mask(train_masks[i*batch_size:i*batch_size+batch_size])
            sentence_masks = sentence_masks.to(device)

            nli_label = batch["nli"]
            nli_label = list(map(lambda x : 1 if x == "Entailment" else 0, nli_label))
            nli_label = np.asarray(nli_label)
            nli_label = torch.from_numpy(nli_label.astype('long'))
            nli_label = nli_label.to(device)
            
            evidence_label = padded_evidence_label.to(device)

            
        
            evidence_out, evidence_preds, evidence_loss, nli_out, nli_loss = \
            model(input_ids, attention_mask,nli_label, evidence_label, sentence_masks= sentence_masks)
            

                
            evidence_loss *= 6. #LOSS RATIO
#             nli_loss *= 2.
            loss = evidence_loss + nli_loss
            
            loss = loss / NUM_ACCUMULATION_STEPS
            try:
                loss.backward()
            except:
                optimizer.zero_grad()
                continue
            
            if ((i + 1) % NUM_ACCUMULATION_STEPS == 0) or (i + 1 == len(train_dataset)):
                optimizer.step()
            
            if i % update_step == update_step - 1:
                print(f'Epoch {epoch}, iter {i}, loss: {round(loss.item(), 4)}')
                
        scheduler.step()

        
        train_score = evaluation(model, train_dataset, train_masks)
        print(f'Epoch {epoch}, train nli f1 p r: %.4f, %.4f, %.4f, evidence f1 p r: %.4f, %.4f, %.4f' % train_score)

        dev_score = evaluation(model, dev_dataset, dev_masks)
        print(f'Epoch {epoch}, dev nli f1 p r: %.4f, %.4f, %.4f, evidence f1 p r: %.4f, %.4f, %.4f' % dev_score)

        dev_perf = dev_score[0] * dev_score[3]
        print(dev_perf)
        if dev_perf >= prev_performance:
            torch.save(model.state_dict(), "checkpoint.model")
            prev_performance = dev_perf
            print("New model saved.")
        else:
            print("Skip saving model.")


In [4]:
train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2680: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
  0%|          | 0/567 [00:00<?, ?it/s]/tmp/ipykernel_622/2740268618.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detac

Epoch 0, iter 199, loss: 0.4733


 71%|███████   | 400/567 [05:06<02:21,  1.18it/s]

Epoch 0, iter 399, loss: 0.9193


100%|██████████| 425/425 [02:32<00:00,  2.78it/s]


...

Epoch 19, iter 399, loss: 0.0389


100%|██████████| 425/425 [02:32<00:00,  2.78it/s]


[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 

  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_622/2740268618.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoded.items()}
/tmp/ipykernel_622/3897857144.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  transformation_indices = [torch.tensor(tensor, device=device) for tensor in transformation_indices]
100%|██████████| 50/50 [00:17<00:00,  2.79it/s]


[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0] [0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,

In [5]:
print(torch.__version__)

2.1.2
